# Spark on Tour
## Ejemplo de uso de API estructurada para generar información de perfil de usuarios

En este notebook vamos a explorar un ejemplo muy representativo de una aplicación de típica de big data: Generar un perfil resumido de los gustos/comportamiento de los usuarios a partir de eventos capturados de su interacción con un sistema.

Concretamente vamos a partir de dos datasets iniciales:
* *movies.csv*: Cada fila representa una película
* *ratings.csv*: Cada fila representa un evento en el que un usuario otorga una puntuación a una película

Vamos a mezclar los datasets y finalmente obtener un único dataset en el que cada fila represente a un usuario y contenga información resumida por cada género como: 
* Número de películas votadas de cada género
* Puntuación media de las pelíclas de cada género

Este es un caso de uso típico por ejemplo para posteriormente:
* Sacar la información a una BBDD operativa y mostrar información estádisca por perfil en un dashboard web
* Entrenar modelos de predicción/segmentación a partir del pefil de los usuarios.

### Importamos librerías, definimos esquemas e inicializamos la sesión Spark.

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


ratingSchema = StructType([
    StructField("user", IntegerType()),
    StructField("movie", IntegerType()),
    StructField("rating", FloatType())
])

movieSchema = StructType([
    StructField("movie", IntegerType()),
    StructField("title", StringType()),
    StructField("genres", StringType())
])


#setup spark session
sparkSession = (SparkSession.builder
                .appName("Introducción API estructurada")
                .master("local[*]")
                .config("spark.scheduler.mode", "FAIR")
                .getOrCreate())
sparkSession.sparkContext.setLogLevel("ERROR")

### Leemos el dataset de ratings usuario / película

In [2]:
ratings = sparkSession.read.csv("/tmp/movielens/ratings.csv", schema=ratingSchema, header=True)
#ratings.show(10)

### Leemos el dataset de películas

In [3]:
movies = sparkSession.read.csv("/tmp/movielens/movies.csv", schema=movieSchema, header=True)
#movies.show(10, truncate=False)

### Transformamos el dataset de películas para asociar cada película con cada uno de sus genéros 

El resultado es un dataset con N filas por película, tantas como género.

In [4]:
movies = movies.select("movie", "title", split("genres", "\|").alias("genres"))
#movies.show(truncate=False)

In [5]:
movies = movies.select("movie", "title", explode("genres").alias("genre"))
#movies.show(10)

### Mezclamos movies y ratings
Enriquecemos la información de ratings con los géneros de cada película, y nos quedaos con un dataframe donde cada película aparece en varias filas, una por cada género

In [6]:
movieRatings = ratings.join(movies, "movie", "left_outer")
#movieRatings.show(10)

### Agregamos por género y usuario
Calculamos indicadores de interés de un usuario en cada género, como el nº total de películas votadas, la media de rating, el máximo rating y el mínimo rating.

Nuestro objetivo es calcular un perfil de usuario por género, por lo que no nos interesan las películas individuales, sino la agregación de los ratings por género para cada usuario

In [7]:
userRatingsGenres = movieRatings.groupBy("user", "genre") \
            .agg( \
                count("rating").alias("num_ratings"),  \
                avg("rating").alias("avg_rating"), \
                min("rating").alias("min_rating"), \
                max("rating").alias("max_rating")) \
            .sort(asc("user"))
#userRatingsGenres.toPandas()

### Generamos el dataset final de perfil de usuario
Ya tenemos la información que queríamos, pero no en la forma que necesitamos para, por ejemplo entrenar un modelo de ML y hacer segmentación de usuarios o predicción de cuanto le va a gusar una película en función de los géneros a los que pertenece.

Necesitamos generar una única fila por cada usuario que represente el perfil del usuario, y por tanto sus 'gustos' con respecto a los diverso géneros de película.

In [8]:
userRatingProfile = userRatingsGenres.groupBy("user") \
                .pivot("genre") \
                .agg(sum("avg_rating").alias("rating"), sum("num_ratings").alias("num"))
userRatingProfile.toPandas()

,user,(no genres listed)_rating,(no genres listed)_num,Action_rating,Action_num,Adventure_rating,Adventure_num,Animation_rating,Animation_num,Children_rating,...,Romance_rating,Romance_num,Sci-Fi_rating,Sci-Fi_num,Thriller_rating,Thriller_num,War_rating,War_num,Western_rating,Western_num
0,1,NaN,NaN,4.322222,90.0,4.388235,85.0,4.689655,29.0,4.547619,...,4.307692,26.0,4.225000,40.0,4.145455,55.0,4.500000,22.0,4.285714,7.0
1,2,NaN,NaN,3.954545,11.0,4.166667,3.0,NaN,NaN,NaN,...,4.500000,1.0,3.875000,4.0,3.700000,10.0,4.500000,1.0,3.500000,1.0
2,3,NaN,NaN,3.571429,14.0,2.727273,11.0,0.500000,4.0,0.500000,...,0.500000,5.0,4.200000,15.0,4.142857,7.0,0.500000,5.0,NaN,NaN
3,4,NaN,NaN,3.320000,25.0,3.655172,29.0,4.000000,6.0,3.800000,...,3.379310,58.0,2.833333,12.0,3.552632,38.0,3.571429,7.0,3.800000,10.0
4,5,NaN,NaN,3.111111,9.0,3.250000,8.0,4.333333,6.0,4.111111,...,3.090909,11.0,2.500000,2.0,3.555556,9.0,3.333333,3.0,3.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606,NaN,NaN,3.178808,151.0,3.503401,147.0,3.714286,42.0,3.448980,...,3.740845,355.0,3.556962,79.0,3.525126,199.0,3.792308,65.0,3.411765,17.0
606,607,NaN,NaN,3.722222,72.0,3.466667,45.0,3.333333,6.0,3.421053,...,3.517241,29.0,3.250000,36.0,4.114754,61.0,4.166667,6.0,4.000000,2.0
607,608,NaN,NaN,3.330325,277.0,3.220994,181.0,3.118182,55.0,2.460227,...,2.886792,106.0,3.296407,167.0,3.536680,259.0,3.578947,19.0,2.636364,11.0
608,609,NaN,NaN,3.090909,11.0,3.200000,10.0,3.000000,1.0,3.000000,...,3.200000,5.0,3.000000,5.0,3.285714,14.0,3.500000,4.0,4.000000,1.0
